# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [662]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [667]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [668]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Sets:**

* I = {Wheat, Corn, Sugar Beets} : Set of crops

**Parameters:**

* A = 500 : Total acres of land available
*  $D_{wheat}$ = 200 : Demand for wheat (tons)
*  $D_{corn}$ = 240 : Demand for corn (tons)
*  $SP_{wheat}$ = 170 : Selling price of wheat ($/ton)
*  $SP_{corn}$ = 150 : Selling price of corn ($/ton)
*  $SP_{beets}$ = 36 : Selling price of sugar beets within quota ($/ton)
*  $SP_{beets\_excess}$ = 10 : Selling price of sugar beets exceeding quota ($/ton)
*  $PP_{wheat}$ = 240 : Purchase price of wheat ($/ton)
*  $PP_{corn}$ = 210 : Purchase price of corn ($/ton)
*  $Q_{beets}$ = 6000 : Quota for sugar beets (tons)
* $Y_{wheat}$ = 2.5 : Yield of wheat (tons/acre)
* $Y_{corn}$ = 3 : Yield of corn (tons/acre)
* $Y_{beets}$ = 20 : Yield of sugar beets (tons/acre)
* $C_{wheat}$ = 150 : Cost of planting wheat ($/acre)
* $C_{corn}$ = 230 : Cost of planting corn ($/acre)
* $C_{beets}$ = 260 : Cost of planting sugar beets ($/acre)

**Decision Variables:**

*  $x_i$ : Acres of land dedicated to crop $i \in I$
*  $p_i$: Tons of crop $i \in \{Wheat, Corn\}$ purchased
*  $s_i$: Tons of crop $i \in I$ sold

**Objective Function:**

Minimize total cost:

 $$ Minimize \sum_{i \in I} C_i \cdot x_i + \sum_{i \in \{Wheat, Corn\}} PP_i \cdot p_i - \sum_{i \in I} SP_i \cdot s_i - SP_{beets\_excess} \cdot max(0, s_{beets} - Q_{beets})$$

**Constraints:**

* Land availability: 
$$\sum_{i \in I} x_i \leq A $$

* Production and purchase meets demand:
$$Y_{wheat} \cdot x_{wheat} + p_{wheat} \geq D_{wheat}$$
$$Y_{corn} \cdot x_{corn} + p_{corn} \geq D_{corn}$$

* Production and sales balance:
$$Y_{wheat} \cdot x_{wheat} - s_{wheat} = D_{wheat}$$
$$Y_{corn} \cdot x_{corn} - s_{corn} = D_{corn}$$
$$Y_{beets} \cdot x_{beets} = s_{beets}$$

* Non-negativity:
$$x_i, p_i, s_i \geq 0,  \forall i \in I$$ 

This model aims to minimize the total costs associated with planting, purchasing, and selling crops. It takes into account the demand for cattle feed, the selling prices of each crop, the yield of each crop per acre, and the costs associated with planting each crop. The model also considers the sugar beet quota imposed by the European Commission and the lower selling price for beets exceeding the quota. 


## 3. Generate the pyomo code

In [669]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [670]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
I = ["Wheat", "Corn", "Sugar Beets"]

# Define parameters (Sample Data)
A = 500
D = {"Wheat": 200, "Corn": 240}
SP = {"Wheat": 170, "Corn": 150, "Sugar Beets": 36}
SP_excess = {"Sugar Beets": 10}  
PP = {"Wheat": 240, "Corn": 210}
Q = {"Sugar Beets": 6000}
Y = {"Wheat": 2.5, "Corn": 3, "Sugar Beets": 20}
C = {"Wheat": 150, "Corn": 230, "Sugar Beets": 260}

# Create a model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land dedicated to each crop
model.p = pyo.Var(["Wheat", "Corn"], domain=pyo.NonNegativeReals) # Tons of wheat/corn purchased
model.s = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold

# Objective function
def objective_rule(model):
    return sum(C[i] * model.x[i] for i in I) + sum(PP[i] * model.p[i] for i in ["Wheat", "Corn"]) - sum(SP[i] * model.s[i] for i in I) - SP_excess["Sugar Beets"] * pyo.max(0, model.s["Sugar Beets"] - Q["Sugar Beets"])

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Land availability
model.land_constraint = pyo.Constraint(expr=sum(model.x[i] for i in I) <= A)

# Production and purchase meets demand
model.demand_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] + model.p["Wheat"] >= D["Wheat"])
model.demand_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] + model.p["Corn"] >= D["Corn"])

# Production and sales balance
model.balance_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] - model.s["Wheat"] == D["Wheat"]) 
model.balance_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] - model.s["Corn"] == D["Corn"])
model.balance_beets = pyo.Constraint(expr=Y["Sugar Beets"] * model.x["Sugar Beets"] == model.s["Sugar Beets"])

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex', etc.
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"Wheat acres: {model.x['Wheat'].value}")
print(f"Corn acres: {model.x['Corn'].value}")
print(f"Sugar Beet acres: {model.x['Sugar Beets'].value}")
print(f"Wheat purchased: {model.p['Wheat'].value}")
print(f"Corn purchased: {model.p['Corn'].value}")
print(f"Wheat sold: {model.s['Wheat'].value}")
print(f"Corn sold: {model.s['Corn'].value}")
print(f"Sugar Beets sold: {model.s['Sugar Beets'].value}")
print(f"Total cost: {model.objective()}")
```


## 4. Input problem data and try running the generated code

In [671]:
import pyomo.environ as pyo

# Define sets
I = ["Wheat", "Corn", "Sugar Beets"]

# Define parameters (Sample Data)
A = 500
D = {"Wheat": 200, "Corn": 240}
SP = {"Wheat": 170, "Corn": 150, "Sugar Beets": 36}
SP_excess = {"Sugar Beets": 10}  
PP = {"Wheat": 240, "Corn": 210}
Q = {"Sugar Beets": 6000}
Y = {"Wheat": 2.5, "Corn": 3, "Sugar Beets": 20}
C = {"Wheat": 150, "Corn": 230, "Sugar Beets": 260}

# Create a model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land dedicated to each crop
model.p = pyo.Var(["Wheat", "Corn"], domain=pyo.NonNegativeReals) # Tons of wheat/corn purchased
model.s = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold

# Objective function
def objective_rule(model):
    return sum(C[i] * model.x[i] for i in I) + sum(PP[i] * model.p[i] for i in ["Wheat", "Corn"]) - sum(SP[i] * model.s[i] for i in I) - SP_excess["Sugar Beets"] * pyo.max(0, model.s["Sugar Beets"] - Q["Sugar Beets"])

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Land availability
model.land_constraint = pyo.Constraint(expr=sum(model.x[i] for i in I) <= A)

# Production and purchase meets demand
model.demand_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] + model.p["Wheat"] >= D["Wheat"])
model.demand_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] + model.p["Corn"] >= D["Corn"])

# Production and sales balance
model.balance_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] - model.s["Wheat"] == D["Wheat"]) 
model.balance_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] - model.s["Corn"] == D["Corn"])
model.balance_beets = pyo.Constraint(expr=Y["Sugar Beets"] * model.x["Sugar Beets"] == model.s["Sugar Beets"])

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex', etc.
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"Wheat acres: {model.x['Wheat'].value}")
print(f"Corn acres: {model.x['Corn'].value}")
print(f"Sugar Beet acres: {model.x['Sugar Beets'].value}")
print(f"Wheat purchased: {model.p['Wheat'].value}")
print(f"Corn purchased: {model.p['Corn'].value}")
print(f"Wheat sold: {model.s['Wheat'].value}")
print(f"Corn sold: {model.s['Corn'].value}")
print(f"Sugar Beets sold: {model.s['Sugar Beets'].value}")
print(f"Total cost: {model.objective()}")

ERROR: Rule failed when generating expression for Objective objective with
index None: AttributeError: module 'pyomo.environ' has no attribute 'max'
ERROR: Constructing component 'objective' from data=None failed:
        AttributeError: module 'pyomo.environ' has no attribute 'max'


AttributeError: module 'pyomo.environ' has no attribute 'max'

## 5. Correct the code to verify model viability (optional)

In [658]:
import pyomo.environ as pyo

# Define sets
I = ["Wheat", "Corn", "Sugar Beets"]

# Define parameters (Sample Data)
A = 500
D = {"Wheat": 200, "Corn": 240}
SP = {"Wheat": 170, "Corn": 150, "Sugar Beets": 36}
SP_excess = {"Sugar Beets": 10}  
PP = {"Wheat": 240, "Corn": 210}
Q = {"Sugar Beets": 6000}
Y = {"Wheat": 2.5, "Corn": 3, "Sugar Beets": 20}
C = {"Wheat": 150, "Corn": 230, "Sugar Beets": 260}

# Create a model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land dedicated to each crop
model.p = pyo.Var(["Wheat", "Corn"], domain=pyo.NonNegativeReals) # Tons of wheat/corn purchased
model.s = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold

# Objective function
def objective_rule(model):
    return sum(C[i] * model.x[i] for i in I) + sum(PP[i] * model.p[i] for i in ["Wheat", "Corn"]) - sum(SP[i] * model.s[i] for i in I) - SP_excess["Sugar Beets"] * pyo.max(0, model.s["Sugar Beets"] - Q["Sugar Beets"])

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Land availability
model.land_constraint = pyo.Constraint(expr=sum(model.x[i] for i in I) <= A)

# Production and purchase meets demand
model.demand_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] + model.p["Wheat"] >= D["Wheat"])
model.demand_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] + model.p["Corn"] >= D["Corn"])

# Production and sales balance
model.balance_wheat = pyo.Constraint(expr=Y["Wheat"] * model.x["Wheat"] - model.s["Wheat"] == D["Wheat"]) 
model.balance_corn = pyo.Constraint(expr=Y["Corn"] * model.x["Corn"] - model.s["Corn"] == D["Corn"])
model.balance_beets = pyo.Constraint(expr=Y["Sugar Beets"] * model.x["Sugar Beets"] == model.s["Sugar Beets"])

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'gurobi', 'cplex', etc.
solver.solve(model)

# Print the results
print("Optimal solution:")
print(f"Wheat acres: {model.x['Wheat'].value}")
print(f"Corn acres: {model.x['Corn'].value}")
print(f"Sugar Beet acres: {model.x['Sugar Beets'].value}")
print(f"Wheat purchased: {model.p['Wheat'].value}")
print(f"Corn purchased: {model.p['Corn'].value}")
print(f"Wheat sold: {model.s['Wheat'].value}")
print(f"Corn sold: {model.s['Corn'].value}")
print(f"Sugar Beets sold: {model.s['Sugar Beets'].value}")
print(f"Total cost: {model.objective()}")

PyomoException: Cannot convert non-constant Pyomo expression (6000  <  20*x3) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [672]:
print(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Sets:**

* I = {Wheat, Corn, Sugar Beets} : Set of crops

**Parameters:**

* A = 500 : Total acres of land available
*  $D_{wheat}$ = 200 : Demand for wheat (tons)
*  $D_{corn}$ = 240 : Demand for corn (tons)
*  $SP_{wheat}$ = 170 : Selling price of wheat ($/ton)
*  $SP_{corn}$ = 150 : Selling price of corn ($/ton)
*  $SP_{beets}$ = 36 : Selling price of sugar beets within quota ($/ton)
*  $SP_{beets\_excess}$ = 10 : Selling price of sugar beets exceeding quota ($/ton)
*  $PP_{wheat}$ = 240 : Purchase price of wheat ($/ton)
*  $PP_{corn}$ = 210 : Purchase price of corn ($/ton)
*  $Q_{beets}$ = 6000 : Quota for sugar beets (tons)
* $Y_{wheat}$ = 2.5 : Yield of wheat (tons/acre)
* $Y_{corn}$ = 3 : Yield of corn (tons/acre)
* $Y_{beets}$ = 20 : Yield of sugar beets (tons/acre)
* $C_{wheat}$ = 150 : Cost of planting wheat ($/acre)
* $C_{corn}$ = 230 : Cost of planting corn ($/acre)
* $C_{beets}$ = 260 : Cost of planting sugar

In [673]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
I = ["Wheat", "Corn", "Sugar Beets"]

# Define parameters (Sample Data)
A = 500
D = {"Wheat": 200, "Corn": 240}
SP = {"Wheat": 170, "Corn": 150, "Sugar Beets": 36}
SP_excess = {"Sugar Beets": 10}  
PP = {"Wheat": 240, "Corn": 210}
Q = {"Sugar Beets": 6000}
Y = {"Wheat": 2.5, "Corn": 3, "Sugar Beets": 20}
C = {"Wheat": 150, "Corn": 230, "Sugar Beets": 260}

# Create a model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(I, domain=pyo.NonNegativeReals) # Acres of land dedicated to each crop
model.p = pyo.Var(["Wheat", "Corn"], domain=pyo.NonNegativeReals) # Tons of wheat/corn purchased
model.s = pyo.Var(I, domain=pyo.NonNegativeReals) # Tons of each crop sold

# Objective function
def objective_rule(model):
    return sum(C[i] * model.x[i] for i in I) + sum(PP[i] * model.p[i] for i in ["Wheat", "Corn"]) - sum(SP[i] * model.s[i] for i in I) - SP_excess["Sugar Beets"] * pyo.max(0, model.s["Sugar Beets"] - Q["Sugar Be